# Port ANCCR function to Python

Utilizing `calculateANCCR` function in `ANCCR.functions.calculateANCCR.m`.

In [5]:
from typing import Dict

import numpy as np

In [187]:
meanITI = 12
numrewards = 500

# anccr model parameters
samplingperiod = 0.2
alpha_anccr = {}
alpha_anccr["exponent"] = 0.1
alpha_anccr["init"] = 0.25
alpha_anccr["min"] = 0.02
alpha_r = 0.2
w = 0.5          
k = 1
minimumrate = 10 ** -3
maximumjitter = 0.1
beta = [1]  # Check what beta does in calculate_anccr
threshold = 0.6
T = meanITI*10  # have very large T

nIter = 100
rwrsp = np.zeros((numrewards,nIter))

In [ ]:
def calculate_anccr(
    eventlog: np.array,
    T: int,
    alpha: Dict,
    k: int,
    samplinginterval: float,
    w: float,
    threshold: float,
    minimumrate: float,
    beta: int,
    alpha_r: float,
    maximumjitter: float,
    optolog=None,
    omidix=[np.nan, np.nan],
    exact_mean_or_not=0,
    nevent_for_edge=0
):
    """
    Calculate ANCCR.
    
    omidx: First entry is omission index, second entry is corresponding reward index.
    exact_mean_or_not: If exact_mean_or_not=1, calculate exact mean for Mij instead of
        using alpha.
    nevent_for_edge: if nevent_for_edge>0, use averaged NC for last nevent to
        calculate edge.
    """
    if alpha_r > 1: alpha_r = 1
    if not optolog:
        optolog = np.zeros((eventlog.shape[0]), 2))  # TODO: What is eventlog.
        
    omtrue = np.zeros(omidx.shape[0], 1)
    uniquetime = np.unique(eventlog[:,1])
    
    ## If more than one event happens at the same time, assume random 
    ## perceptual delay between them.
    
    for jt in range(1, len(uniquetime)):
        if sum(eventlog[:, 1] == uniquetime[jt]) == 1:
            continue
        idx = eventlog[:, 1] == uniquetime(jt)
        idx = idx[1:]
        eventlog[idx, 1] = eventlog[idx, 1] + np.randn(len(idx), 1) * maximumjitter
        
# TODO: Sort rows of eventlog by time (2 column)
eventlog = sortrows(eventlog,2);
ntime = eventlog.shape[0]

In [140]:
def legacy_simulate_background_reward(numrewards, rewITI, rewlabel, rewmag, truncation):
    """
    Output an eventlog with exclusively background
    rewards. Inputs may be scalar or arrays depending on the number of unique
    background rewards specified.
    """
    # Set upper limit on ITI (only used if truncation == 1)
    maxrewITI = 3 * rewITI
    # Initialize empty eventlog
    eventlog = np.zeros((numrewards,3))
    # Keep track of reward indices
    running_idx = 0
    # If multiple unique rewards, loop instance for each type
    for irw in range(1, rewlabel+1):
        # Start simulation time at 0
        running_time = 0
        # For each reward (irw) simulate 'numrewards' timesteps
        for i in range(1, numrewards+1):
            # Generate exp. dist. timestep for current reward instance
            new_ts = round(np.random.exponential(rewITI), 4)
            if truncation:  # If capping ITI at maxITI
                while new_ts > maxrewITI:
                    # Simulate a new ts if maxITI is exceeded
                    new_ts = round(np.random.exponential(rewITI), 4)
            # Update eventlog
            eventlog[running_idx, 0] = rewlabel  # Column 1 = label
            eventlog[running_idx, 1] = new_ts + running_time  # " 2 = ts
            eventlog[running_idx, 2] = rewmag  # " 3 = rew. magnitude
            running_idx = running_idx + 1
            running_time = running_time + new_ts  # Update running time
    
    # Resort eventlog by timestamp
    eventlog = eventlog[eventlog[:, 1].argsort()]
    
    # Once any reward reaches to numrewards, finish the session
    # lasttrial = min(cellfun(@(x) find(eventlog(:,1)==x,1,'last'),num2cell(rewlabel)));
    # eventlog(lasttrial+1:end,:) = [];
    
    return eventlog

In [171]:
def simulate_background_reward(numrewards, rewITI, rewlabel, rewmag, truncation):
    """
    Output an eventlog with exclusively background
    rewards. Inputs may be scalar or arrays depending on the number of unique
    background rewards specified.
    """
    labels = np.array([rewlabel] * numrewards)
    rewards = np.array([rewmag] * numrewards)
    ts = np.random.exponential(rewITI, numrewards).cumsum().round(4)
    eventlog = np.array([labels, ts, rewards]).T
    
    return eventlog

In [172]:
def test_simulate_background_reward():
    expected = np.array([
        [1] * 10,
        [
            2.4589,
            3.6462,
            28.4103,
            29.4976,
            34.9972,
            62.9270,
            78.2671,
            85.5094,
            86.0305,
            86.4594,
        ],
        [1] * 10]
    ).T
    actual = simulate_background_reward(10, 12, 1, 1, 0)
    
    assert expected.shape == actual.shape, f"{actual.shape} is not {expected.shape}"
    
test_simulate_background_reward()

In [177]:
eventlog = simulate_background_reward(5, 12, 1, 1, 0)
eventlog

array([[ 1.    ,  0.8565,  1.    ],
       [ 1.    , 21.8715,  1.    ],
       [ 1.    , 27.7139,  1.    ],
       [ 1.    , 31.841 ,  1.    ],
       [ 1.    , 32.1078,  1.    ]])

In [180]:
eventlog[:,1] + 1

array([ 1.8565, 22.8715, 28.7139, 32.841 , 33.1078])